# TP1
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 3.

Use o “package” Cryptography para
    1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
    2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.


In [1]:
!pip install cryptography

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import os

In [3]:
chave = os.urandom(32)
nonce = os.urandom(16)

In [4]:
mensagem = b"Grupo 17"
salt = os.urandom(16)

In [5]:
kdf = PBKDF2HMAC(
    algorithm = hashes.SHA256(),
    length = 32,
    salt = salt,
    iterations = 10000,
    backend = default_backend()
)

chave = kdf.derive(mensagem)

In [6]:
cifra = Cipher(algorithms.AES(chave), modes.GCM(nonce), backend = default_backend())

In [7]:
aead = cifra.encryptor()
plaintext = b"Grupo 17"
textoCifrado = aead.update(plaintext) + aead.finalize()
tag = aead.tag

In [8]:
cifra = Cipher(
    algorithms.AES(chave), 
    modes.GCM(nonce, tag), 
    backend = default_backend()
)
aead = cifra.decryptor()
decrypted_plaintext = aead.update(textoCifrado) + aead.finalize()

print(decrypted_plaintext == plaintext)

True


In [11]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.tweakable_block_ciphers import TweakableBlockCipher
from cryptography.hazmat.primitives.tweakable_block_ciphers.algorithms import AES, ChaCha20
from cryptography.hazmat.primitives.tweakable_block_ciphers.modes import XTS
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidTag

# Define the key and IV
key = b'0123456789abcdef0123456789abcdef'
iv = b'0123456789abcdef'

# Create an instance of the primitive block cipher
cipher = AES(key)
# cipher = ChaCha20(key)

# Create an instance of the tweakable block cipher with XTS mode
tbc = TweakableBlockCipher(cipher, XTS(iv))

# Define the message and AAD
message = b'This is a secret message.'
aad = b'This is additional data.'

# Define the tweak value
tweak = 0

# Create an AEAD object with the tweakable block cipher and tweak value
aead = tbc.encryptor(tweak)

# Encrypt the message and calculate the tag
ciphertext = aead.update(message) + aead.finalize()
tag = aead.tag

# Decrypt the message and verify the tag
aead = tbc.decryptor(tweak, tag)
plaintext = aead.update(ciphertext) + aead.finalize()

try:
    aead.verify(tag)
    print('Tag is valid.')
except InvalidTag:
    print('Tag is invalid.')


ModuleNotFoundError: No module named 'cryptography.hazmat.primitives.tweakable_block_ciphers'